# Projet Titanic

Voici une base de donnée traité avec MongoDB sur le Titanic aka Le RMS Titanic qui est un paquebot transatlantique britannique qui fait naufrage dans l'océan Atlantique Nord en 1912 à la suite d'une collision avec un iceberg, lors de son voyage inaugural de Southampton à New York.

# C'est parti

In [1]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [2]:
client = pymongo.MongoClient()
client = MongoClient("mongo")
database = client['exercices']
collection = database['titanic']

['admin', 'config', 'exercices', 'local', 'series', 'yt']


In [40]:
print(client.database_names())

['admin', 'config', 'exercices', 'local', 'series', 'titanic', 'yt']


In [9]:
df_titanic = pd.read_csv("./data/titanicmongolouis.csv")
df_titanic.head()

,ID,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,1,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,2,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,3,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,4,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,5,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


## Question 0

### Netoyer les données

In [6]:
df_titanic = pd.read_csv("./data/titanicmongolouis.csv")

In [7]:
df_titanic.rename(columns={'ID':'_id'}, inplace=True)
l_d =df_titanic.to_dict(orient='records')

### Importer les données

In [8]:
df_titanic

,_id,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,1,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,2,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,3,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,4,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,5,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
5,6,0,3,Mr. James Moran,male,27.0,0,0,8.4583
6,7,0,1,Mr. Timothy J McCarthy,male,54.0,0,0,51.8625
7,8,0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.0750
8,9,1,3,Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson,female,27.0,0,2,11.1333
9,10,1,2,Mrs. Nicholas (Adele Achem) Nasser,female,14.0,1,0,30.0708


In [10]:
from pymongo import MongoClient

client = MongoClient("mongo:27017")

db_titanic = client.titanic

collection_titanic = db_titanic['passenger']

collection_titanic.delete_many({})

collection_titanic.insert_many(l_d)

## 1 • Nombre hommes et de femmes à bord

In [22]:
cur = collection_titanic.find({"Sex":"male"})
cur.count()

573

In [23]:
curr = collection_titanic.find({"Sex":"female"})
curr.count()

314

In [51]:
cur = collection_titanic.aggregate([
    { "$group": { 
        "_id": "$Sex",
        "count": { "$sum": 1 } 
        }  
    }
])
list(cur)

[{'_id': 'male', 'count': 573}, {'_id': 'female', 'count': 314}]

## 2 • Nombre d'enfants à bord

In [69]:
cur = collection_titanic.find({"$and":[{"Age":{"$lt": 18}}]}, {"Age":1})
cur.count()

130

## 3 • Moyenne d'âge

In [83]:
result = collection_titanic.aggregate([{
    "$group": {
        "_id": "$None",
        "moyenne_age_general": {"$avg": "$Age"}
    }
}])

for line in result:
    print(line)

{'_id': None, 'moyenne_age_general': 29.471443066516347}


In [82]:
result = collection_titanic.aggregate([{
    "$group": {
        "_id": "$Sex",
        "moyenne_age_par_sexe": {"$avg": "$Age"}
    }
}])

for line in result:
    print(line)

{'_id': 'male', 'moyenne_age_par_sexe': 30.4313612565445}
{'_id': 'female', 'moyenne_age_par_sexe': 27.719745222929937}


## 4 • Les 10 passagers les plus riches

In [102]:
result = collection_titanic.aggregate([{
    "$group": {
        "_id": "$Name",
        "prix_entree_plus_cher": {"$avg": "$Fare"}
    }
},

    { "$sort": { "prix_entree_plus_cher": -1 } },
    { "$limit": 10 }])

list(result)

[{'_id': 'Mr. Thomas Drake Martinez Cardeza',
  'prix_entree_plus_cher': 512.3292},
 {'_id': 'Miss. Anna Ward', 'prix_entree_plus_cher': 512.3292},
 {'_id': 'Mr. Gustave J Lesurer', 'prix_entree_plus_cher': 512.3292},
 {'_id': 'Miss. Alice Elizabeth Fortune', 'prix_entree_plus_cher': 263.0},
 {'_id': 'Mr. Mark Fortune', 'prix_entree_plus_cher': 263.0},
 {'_id': 'Miss. Mabel Helen Fortune', 'prix_entree_plus_cher': 263.0},
 {'_id': 'Mr. Charles Alexander Fortune', 'prix_entree_plus_cher': 263.0},
 {'_id': 'Miss. Emily Borie Ryerson', 'prix_entree_plus_cher': 262.375},
 {'_id': 'Miss. Susan Parker Ryerson', 'prix_entree_plus_cher': 262.375},
 {'_id': 'Mrs. James (Helene DeLaudeniere Chaput) Baxter',
  'prix_entree_plus_cher': 247.5208}]

## 5 • Combien 1 = Survécu / 0 = disparu du Titanic

In [110]:
cur = collection_titanic.aggregate([
    { "$group": { 
        "_id": "$Survived",
        "count": { "$sum": 1 } 
        }  
    }
])
list(cur)

[{'_id': 1, 'count': 342}, {'_id': 0, 'count': 545}]

## 6 • Combien avait des parents ou enfants à bord et ont survécu

In [117]:
cur = collection_titanic.find({"$and":[{"Parents/Children Aboard":{"$gt": 0}}, {"Survived":1}]})
cur.count()

109

## 7 • Combien de 1ere/2eme/3eme classe ont survécu

In [125]:
cur = collection_titanic.find({"$and":[{"Pclass":{"$eq": 1}}, {"Survived":1}]})
cur.count()

136

In [119]:
cur = collection_titanic.find({"$and":[{"Pclass":{"$eq": 2}}, {"Survived":1}]})
cur.count()

87

In [120]:
cur = collection_titanic.find({"$and":[{"Pclass":{"$gte": 3}}, {"Survived":1}]})
cur.count()

119

## 8 • La famille "Fortune" à bord

In [136]:
cur = collection_titanic.find({"Name":{'$regex' : 'Fortune', '$options' : 'i'}})

list(cur)

[{'_id': 28,
  'Survived': 0,
  'Pclass': 1,
  'Name': 'Mr. Charles Alexander Fortune',
  'Sex': 'male',
  'Age': 19.0,
  'Siblings/Spouses Aboard': 3,
  'Parents/Children Aboard': 2,
  'Fare': 263.0},
 {'_id': 88,
  'Survived': 1,
  'Pclass': 1,
  'Name': 'Miss. Mabel Helen Fortune',
  'Sex': 'female',
  'Age': 23.0,
  'Siblings/Spouses Aboard': 3,
  'Parents/Children Aboard': 2,
  'Fare': 263.0},
 {'_id': 340,
  'Survived': 1,
  'Pclass': 1,
  'Name': 'Miss. Alice Elizabeth Fortune',
  'Sex': 'female',
  'Age': 24.0,
  'Siblings/Spouses Aboard': 3,
  'Parents/Children Aboard': 2,
  'Fare': 263.0},
 {'_id': 436,
  'Survived': 0,
  'Pclass': 1,
  'Name': 'Mr. Mark Fortune',
  'Sex': 'male',
  'Age': 64.0,
  'Siblings/Spouses Aboard': 1,
  'Parents/Children Aboard': 4,
  'Fare': 263.0}]